In [19]:
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# print(plotly.__version__) # requires version >= 1.9.0
init_notebook_mode(connected=True);

import pandas as pd
# import cufflinks as cf
#cf.set_config_file(offline=True, world_readable=True, theme='ggplot')

df = pd.read_csv('cbo.csv')
df.columns[1:];
list(df['Income']);

df2 = pd.DataFrame(data=df.iloc[:,1:].transpose(), index=df.columns[1:])
df2 = df2/1000 # convert millions USD to billions USD
df2.columns=list(df['Income']) # like a transpose?

I recently saw a CBO table of change in tax revenues by income level and by year.  I think this is interesting data and ought to be visualized.  

The original article I read is [here (Washington Post).](https://www.washingtonpost.com/news/wonk/wp/2017/11/26/senate-gop-tax-bill-hurts-the-poor-more-than-originally-thought-cbo-finds/)

The CBO report is [here (cbo.gov)](https://www.cbo.gov/publication/53348)

The original table is in units of **Millions** USD and I've converted the units to **Billions** USD. Here's the table in **Billions** USD:

In [70]:
df2

,0 - 10k,10k - 20k,20k - 30k,30k - 40k,40k - 50k,50k - 75k,75k - 100k,100k - 200k,200k - 500k,500k - 1M,1M and above,All
2019,1.54,0.96,0.08,-3.92,-6.04,-22.27,-21.52,-64.24,-59.57,-24.88,-34.10,-233.95
2021,5.87,9.05,9.00,0.77,-2.66,-19.47,-21.26,-63.99,-60.11,-24.08,-28.69,-195.57
2023,7.44,11.40,10.20,2.44,-1.80,-16.94,-18.47,-52.90,-50.01,-18.69,-13.10,-140.40
2025,8.68,12.18,12.21,2.56,-1.53,-17.38,-19.54,-55.47,-54.53,-20.00,-15.81,-148.62
2027,10.07,16.06,16.72,7.61,5.27,3.98,-1.39,-5.34,-5.19,-1.94,-5.78,40.11


Here's the data plotted with the year on the x-axis and the amounts (in millions of USD) of revenue change for each income group.  Positive means that group is paying more taxes (as compared to <FIXME YEAR>) and negative means that group is paying less.

In [42]:
clist=['blue', 'red', 'green', 'pink', 'orange'] # colors
slist=['circle', 'square', 'triangle-up'] # symbols
# markers
mlist = [dict(color=c, symbol=s, size=14) for s in slist for c in clist]

iplot([{
    'x': df2.index,
    'y': df2[col],
    'name': col,
    'mode': 'lines+markers',
    'marker': mlist[idx]
}  for idx,col in enumerate(df2.columns[0:-1])], filename='cufflinks/simple-line')

It looks like there are roughly three groups:
* \$0 - \$50k
* \$50k - \$100k + \$500k and above
* \$100k - \$500k

If we look at the cumulative sum (across the years) we get the following table:

In [72]:
df2.cumsum()

,0 - 10k,10k - 20k,20k - 30k,30k - 40k,40k - 50k,50k - 75k,75k - 100k,100k - 200k,200k - 500k,500k - 1M,1M and above,All
2019,1.54,0.96,0.08,-3.92,-6.04,-22.27,-21.52,-64.24,-59.57,-24.88,-34.10,-233.95
2021,7.41,10.01,9.08,-3.15,-8.70,-41.74,-42.78,-128.23,-119.68,-48.96,-62.79,-429.52
2023,14.85,21.41,19.28,-0.71,-10.50,-58.68,-61.25,-181.13,-169.69,-67.65,-75.89,-569.92
2025,23.53,33.59,31.49,1.85,-12.03,-76.06,-80.79,-236.60,-224.22,-87.65,-91.70,-718.54
2027,33.60,49.65,48.21,9.46,-6.76,-72.08,-82.18,-241.94,-229.41,-89.59,-97.48,-678.43


In [73]:
iplot([{
    'x': df2.index,
    'y': df2.cumsum()[col],
    'name': col,
    'mode': 'lines+markers',
    'marker': mlist[idx]
}  for idx,col in enumerate(df2.cumsum().columns[0:-1])], filename='cufflinks/simple-line')

This seems to 'smooth out' the clustering of the 3 groups, which makes sense given that a cumulative sum is inherently some form of a 'low-pass-filter'.  What about a bar chart of the final row of the cumulative sum?  I'm glad I asked.

In [74]:
iplot([go.Bar(
    x = df2.columns[0:-1],
    y = list(df2.cumsum().iloc[-1, 0:-1]))])

If this is a game, then \$100k - \$500k certainly appear to be the winners.  But presumably, there are different numbers of people in eache income bracket.  What if we simply divide our data by the population of each income bracket, and normalize this data, per person. I found population information as a function of income bracket from the [2016 American Community Survey's Public Use Microdata Sample.](https://www.census.gov/programs-surveys/acs/data/pums.html)

After putting that data in to the same bins as the CBO table above, we can see the population of each income bracket in units of <FIXME household/people>:

In [76]:
pa = pd.read_csv('ss16pusa.csv')
#dfpop = df.read_csv('FIXME')

In [86]:
# iplot([go.Histogram(
ranges = [0,10,20,30,40,50,75,100,200,500,10000,1e15]
ranges = zip(ranges[0:-1], ranges[1:])
df.groupby(pd.cut(df.a, ranges)).count()
type(pa['ADJINC'])
# )])

pandas.core.series.Series